<a href="https://colab.research.google.com/github/Gregrs400/cmpsc472Project2/blob/main/cmpsc472Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
%%writefile cmpsc472Project2.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>
#include <sys/wait.h>
#include <sys/time.h>
#include <sys/resource.h>
#include <time.h>
#include <stdbool.h>
#include <semaphore.h>
#include <pthread.h>

#define MAX_COLUMNS 30
#define GLOBAL_RESOURCES 10

sem_t semaphore;

struct Landscape {
    int rows;
    int columns;
    int (*grid)[MAX_COLUMNS];
};

void printLandscape(struct Landscape *landscape);

struct Fire {
  int row;
  int col;
};

struct Resource {
    int currentRow;
    int currentCol;
    int assignedFireRow;
    int assignedFireCol;
};

struct Resource freeResources[GLOBAL_RESOURCES];
struct Resource assignedResources[GLOBAL_RESOURCES];
void extinguishFire(struct Resource *resource, struct Landscape *landscape);
void moveLeft(struct Resource *resource, struct Landscape *landscape);
void moveRight(struct Resource *resource, struct Landscape *landscape);
void moveUp(struct Resource *resource, struct Landscape *landscape);
void moveDown(struct Resource *resource, struct Landscape *landscape);

struct ThreadArgs {
  int beginRow;
  int endRow;
  // Pipe array for IPC
  int fireToDispatch;
  struct Landscape *landscape;
};

void initResourceArrays()
{

    for (int i = 0; i < GLOBAL_RESOURCES; i++)
    {

        freeResources[i].currentRow = -1;
        freeResources[i].currentCol = -1;
        freeResources[i].assignedFireRow = -1;
        freeResources[i].assignedFireCol = -1;
        assignedResources[i].currentRow = -1;
        assignedResources[i].currentCol = -1;
        assignedResources[i].assignedFireRow = -1;
        assignedResources[i].assignedFireCol = -1;

    }

}

void* searchForFires(void* args) {
  struct ThreadArgs* threadArgs = (struct ThreadArgs*)args;
  int beginRow = threadArgs->beginRow;
  int endRow = threadArgs->endRow;
  int fireToDispatch = threadArgs->fireToDispatch;
  struct Landscape *landscape = threadArgs->landscape;
  int foundFireCoords[2];
  // Lock semaphore before writing to parent process
  sem_wait(&semaphore);
  for (int i = beginRow; i < endRow; i++) {
    for (int j = 0; j < landscape->columns; j++) {
      if (landscape->grid[i][j] == 1) {
        foundFireCoords[0] = i;
        foundFireCoords[1] = j;
        write(fireToDispatch, &foundFireCoords, sizeof(foundFireCoords));
      }
    }
  }
  // Unlock semaphore after writing to parent process
  sem_post(&semaphore);

}

// Function to set a random position in the array to 1, which represents a wildfire.
struct Fire setFire(struct Landscape *landscape) {
  struct Fire fire;
  // Seed the random number generator
  srand(time(NULL));
  int numOfRows = landscape -> rows;
  int numOfColumns= landscape -> columns;
  do
  {
  // Randomly select a row
  fire.row = rand() % (numOfRows);
  // Randomly select a column
  fire.col = rand() % (numOfColumns);
  }while (landscape -> grid[fire.row][fire.col] != 0);

  // Set the value at the selected row and column to 1
  landscape -> grid[fire.row][fire.col] = 1;
  printf("new fire at position (%d,%d)\n", fire.row, fire.col);
  return fire;
}

void placeResources(struct Landscape *landscape)
{

  // Seed the random number generator
  srand(time(NULL));

  int numOfRows = landscape -> rows;
  int numOfColumns= landscape -> columns;

  int resourceRow = 0;
  int resourceCol = 0;

  for (int i = 0; i < GLOBAL_RESOURCES; i++)
  {

    do
    {

        resourceRow = rand() % (numOfRows) + 0;
        resourceCol = rand() % (numOfColumns - 0) + 0;

    }while (landscape -> grid[resourceRow][resourceCol] != 0);

    landscape -> grid[resourceRow][resourceCol] = 7;
    freeResources[i].currentRow = resourceRow;
    freeResources[i].currentCol= resourceCol;
    printf("new resource at position (%d,%d)\n", resourceRow, resourceCol);
  }

}

// Prints out the entire landscape array.

void printLandscape(struct Landscape *landscape) {
  for (int i = 0; i < landscape->rows; i++) {
    for (int j = 0; j < landscape->columns; j++) {
      if (landscape -> grid[i][j] == 1) // fire
      {
        printf("\033[1;91m%d\033[0m ", landscape->grid[i][j]);
        continue;
      }
      else if (landscape -> grid[i][j] == 0) // ground
      {
        printf("\033[1;32m%d\033[0m ", landscape->grid[i][j]);
        continue;
      }
      else if (landscape -> grid[i][j] == 5) // 911 Center
      {
        printf("\033[1;94m%d\033[0m ", landscape->grid[i][j]);
        continue;
      }
      else if (landscape -> grid[i][j] == 7) // Firefighting resource
      {
        printf("\033[1;93m%d\033[0m ", landscape->grid[i][j]);
        continue;
      }
      printf("%d ", landscape->grid[i][j]);
    }
    // Print a newline at the end of each row.
    printf("\n");
  }
}

void spiralScanForFreeResource(struct Landscape *landscape, int row, int col, int *result)
{

    int bound = 0;

    if ((landscape -> rows) - row > row)
    {

        bound = (landscape -> rows) - row;

    }
    else
    {

        bound = row;

    }
    for(int i = 1; i < bound+1; i++)
    {

        for (int j = row - i; j <= row + i; j++)
        {

            for (int k = col - i; k <= col + i; k++)
            {

                if (j >= landscape -> rows || j < 0 || k >= landscape -> columns || k < 0)
                {
                    continue;
                }
                else if (j > row - i && j < row + i && k > col - i &&  k < col + i)
                {
                    continue;
                }
                else
                {
                     if (landscape -> grid[j][k] == 7)
                     {

                        for (int i = 0; i < GLOBAL_RESOURCES; i++)
                        {

                            if (freeResources[i].currentRow == j && freeResources[i].currentCol == k)
                            {

                                result[0] = j;
                                result[1] = k;
                                return;

                            }

                        }

                     }

                }

            }

        }

    }

}

void moveResource(struct Resource *resource, struct Landscape *landscape)
{

    int rowDifference = resource -> assignedFireRow - resource -> currentRow;
    int colDifference = resource -> assignedFireCol - resource -> currentCol;

    printf("(%d, %d) Row Difference: %d Column Difference: %d\n", resource -> currentRow, resource -> currentCol, rowDifference, colDifference);

    // if resource is within 2 rows and columns of fire
    if (rowDifference <= 2 && rowDifference >= -2)
    {
        if (colDifference <= 2 && colDifference >= -2)
        {
            extinguishFire(resource, landscape);
            return;
        }
    }
    // determine where Resource is
    if (resource -> currentCol == 0)  // if resource currentCol is 0
    {
        if (resource -> currentRow == 0)  // if resource currentRow is 0
        {
            printf("\ntop left corner\n"); // top left corner
            // possible movements: right, down
            if (colDifference > rowDifference)  // if fire is more right than down
            {
                if (landscape -> grid[resource -> currentRow][resource -> currentCol+1] != 0)  // if element to the right is not a 0
                {
                    if (landscape -> grid[resource -> currentRow+1][resource -> currentCol] != 0)  // if element below is not a 0
                    {
                        printf("stuck");  // stuck
                        return;
                    }
                    moveDown(resource, landscape);  // move down 1
                    return;
                }
                moveRight(resource, landscape);  // move right 1
                return;
            }
            else if(rowDifference >= colDifference)  // else if fire is more down than right
            {
                if (landscape -> grid[resource -> currentRow+1][resource -> currentCol] != 0)  // if element below is not a 0
                {
                    if (landscape -> grid[resource -> currentRow][resource -> currentCol+1] != 0)  // if element to the right is not a 0
                    {
                        printf("stuck");  // stuck
                        return;
                    }
                    moveRight(resource, landscape);  // move right 1
                    return;
                }
                moveDown(resource, landscape);  // move down 1
                return;
            }
        }
        else if (resource -> currentRow == landscape -> rows - 1)  // else if resource currentRow is landscape.rows - 1
        {
            printf("\nbottom left corner\n"); // bottom left corner
            // possible movements: right, up
            if ( colDifference > (-1 * rowDifference))  // if fire is more right than up
            {
                if (landscape -> grid[resource -> currentRow][resource -> currentCol+1] != 0)  // if element to the right is not a 0
                {
                    if (landscape -> grid[resource -> currentRow-1][resource -> currentCol] != 0)  // if element above is not a 0
                    {
                        printf("stuck");  // stuck
                        return;
                    }
                    moveUp(resource, landscape);  // move up 1
                    return;
                }
                moveRight(resource, landscape);  // move right 1
                return;
            }
            else if ((-1 * rowDifference) >= colDifference)  // else if fire is more up than right
            {
                if (landscape -> grid[resource -> currentRow-1][resource -> currentCol] != 0)  // if element above is not a 0
                {
                    if (landscape -> grid[resource -> currentRow][resource -> currentCol+1] != 0)  // if element to the right is not a 0
                    {
                        printf("stuck");  // stuck
                        return;
                    }
                     moveRight(resource, landscape);  // move right 1
                     return;
                }
                moveUp(resource, landscape);  // move up 1
                return;
            }
        }
        else // else
        {
            printf("\nleft edge\n");  // left edge
            // possible movements: up, down, right
            if ( colDifference > (-1 * rowDifference))  // if fire is more right than up
            {
                if (landscape -> grid[resource -> currentRow][resource -> currentCol+1] != 0)  // if element to the right is not a 0
                {
                    if (landscape -> grid[resource -> currentRow-1][resource -> currentCol] != 0)  // if element above is not a 0
                    {
                        printf("stuck");  // stuck
                        return;
                    }
                    moveUp(resource, landscape);  // move up 1
                    return;
                }
                moveRight(resource, landscape);  // move right 1
                return;
            }
            else if ((-1 * rowDifference) >= colDifference)  // else if fire is more up than right
            {
                if (landscape -> grid[resource -> currentRow-1][resource -> currentCol] != 0)  // if element above is not a 0
                {
                    if (landscape -> grid[resource -> currentRow][resource -> currentCol+1] != 0)  // if element to the right is not a 0
                    {
                        printf("stuck");  // stuck
                        return;
                    }
                    moveRight(resource, landscape);  // move right 1
                    return;
                }
                moveUp(resource, landscape);  // move up 1
                return;
            }
            else if (colDifference > rowDifference)  // else if fire is more right than down
            {
                if (landscape -> grid[resource -> currentRow][resource -> currentCol+1] != 0)  // if element to the right is not a 0
                {
                    if (landscape -> grid[resource -> currentRow+1][resource -> currentCol] != 0)  // if element below is not a 0
                    {
                        printf("stuck");  // stuck
                        return;
                    }
                    moveDown(resource, landscape);  // move down 1
                    return;
                }
                moveRight(resource, landscape);  // move right 1
                return;
            }
            else if(rowDifference >= colDifference)  // else if fire is more down than right
            {
                if (landscape -> grid[resource -> currentRow+1][resource -> currentCol] != 0)  // if element below is not a 0
                {
                    if (landscape -> grid[resource -> currentRow][resource -> currentCol+1] != 0)  // if element to the right is not a 0
                    {
                        printf("stuck");  // stuck
                        return;
                    }
                    moveRight(resource, landscape);  // move right 1
                    return;
                }
                moveDown(resource, landscape);  // move down 1
                return;
            }
        }
    }
    else if (resource -> currentCol == landscape -> columns - 1)  // else if resource currentCol is landscape.columns - 1
    {
        if (resource -> currentRow == 0)  // if resource currentRow is 0
        {
                printf("\ntop right corner\n");  // top right corner
                // possible movements: left, down
               if ((-1 * colDifference) > rowDifference)  // if fire is more left than down
               {
                    if (landscape -> grid[resource -> currentRow][resource -> currentCol-1] != 0)  // if element to the left is not a 0
                    {
                        if (landscape -> grid[resource -> currentRow+1][resource -> currentCol] != 0)  // if element below is not a 0
                        {
                            printf("stuck");  // stuck
                            return;
                        }
                        moveDown(resource, landscape);  // move down 1
                        return;
                    }
                    moveLeft(resource, landscape);  // move left 1
                    return;
                }
                else if (rowDifference >= (-1 * colDifference))  // else if fire is more down than left
                {
                    if (landscape -> grid[resource -> currentRow+1][resource -> currentCol] != 0)  // if element below is not a 0
                    {
                        if (landscape -> grid[resource -> currentRow][resource -> currentCol-1] != 0)  // if element to the left is not a 0
                        {
                            printf("stuck");  // stuck
                            return;
                        }
                        moveLeft(resource, landscape);  // move left 1
                        return;
                    }
                    moveDown(resource, landscape);  // move down 1
                    return;
                }
            }
            else if (resource -> currentRow == landscape -> rows - 1)  // else if resource currentRow is landscape.rows -1
            {
                printf("\nbottom right corner\n");  // bottom right corner
                // possible movements: left, up
                if ((-1 * colDifference) > (-1 * rowDifference))  // if fire is more left than up
                {
                    if (landscape -> grid[resource -> currentRow][resource -> currentCol-1] != 0)  // if element to the left is not a 0
                    {
                        if (landscape -> grid[resource -> currentRow-1][resource -> currentCol] != 0)  // if element above is not a 0
                        {
                            if (landscape -> grid[resource -> currentRow+1][resource -> currentCol] != 0)  // if element below is not a 0
                            {
                                printf("stuck");  // stuck
                                return;
                            }
                            moveDown(resource, landscape);  // move down 1
                            return;
                        }
                        moveUp(resource, landscape);  // move up 1
                        return;
                    }
                    moveLeft(resource, landscape);  // move left 1
                    return;
                }
                else if ((-1 * rowDifference) >= (-1 * colDifference))  // else if fire is more up than left
                {
                    if (landscape -> grid[resource -> currentRow-1][resource -> currentCol] != 0)  // if element above is not a 0
                    {
                        if (landscape -> grid[resource -> currentRow][resource -> currentCol-1] != 0)  // if element to the left is not a 0
                        {
                            if (landscape -> grid[resource -> currentRow+1][resource -> currentCol] != 0)  // if element below is not a 0
                            {
                                printf("stuck");  // stuck
                                return;
                            }
                            moveDown(resource, landscape);  // move down 1
                            return;
                        }
                        moveLeft(resource, landscape);  // move left 1
                        return;
                    }
                    moveUp(resource, landscape);  // move up 1
                    return;
                }
            }
            else // else
            {
                printf("\nright edge\n");  // right edge
                // possible movements: up, down, left
                if ((-1 * colDifference) > (-1 * rowDifference))  // if fire is more left than up
                {
                    if (landscape -> grid[resource -> currentRow][resource -> currentCol-1] != 0)  // if element to the left is not a 0
                    {
                         if (landscape -> grid[resource -> currentRow-1][resource -> currentCol] != 0)  // if element above is not a 0
                         {
                            if (landscape -> grid[resource -> currentRow+1][resource -> currentCol] != 0)  // if element below is not a 0
                            {
                                printf("stuck");  // stuck
                                return;
                            }
                            moveDown(resource, landscape);  // move down 1
                            return;
                         }
                        moveUp(resource, landscape);  // move up 1
                        return;
                    }
                    moveLeft(resource, landscape);  // move left 1
                    return;
                }
                else if ((-1 * rowDifference) >= (-1 * colDifference))  // else if fire is more up than left
                {
                     if (landscape -> grid[resource -> currentRow-1][resource -> currentCol] != 0)  // if element above is not a 0
                     {
                        if (landscape -> grid[resource -> currentRow][resource -> currentCol-1] != 0)  // if element to the left is not a 0
                        {
                            if (landscape -> grid[resource -> currentRow+1][resource -> currentCol] != 0)  // if element below is not a 0
                            {
                                printf("stuck");  // stuck
                                return;
                            }
                            moveDown(resource, landscape);  // move down 1
                            return;
                        }
                       moveLeft(resource, landscape);  // move left 1
                       return;
                     }
                    moveUp(resource, landscape);  // move up 1
                    return;
                }
                else if ((-1 * colDifference) > rowDifference)  // else if fire is more left than down
                {
                    if (landscape -> grid[resource -> currentRow][resource -> currentCol-1] != 0)  // if element to the left is not a 0
                    {
                        if (landscape -> grid[resource -> currentRow+1][resource -> currentCol] != 0)  // if element below is not a 0
                        {
                            if (landscape -> grid[resource -> currentRow-1][resource -> currentCol] != 0)  // if element above is not a 0
                            {
                                printf("stuck");  // stuck
                                return;
                            }
                            moveUp(resource, landscape);  // move up 1
                            return;
                        }
                        moveDown(resource, landscape);  // move down 1
                        return;
                    }
                    moveLeft(resource, landscape);  // move left 1
                    return;
                }
                else if (rowDifference >= (-1 * colDifference))  // else if fire is more down than left
                {
                    if (landscape -> grid[resource -> currentRow+1][resource -> currentCol] != 0)  // if element below is not a 0
                    {
                        if (landscape -> grid[resource -> currentRow][resource -> currentCol-1] != 0)  // if element to the left is not a 0
                        {
                            if (landscape -> grid[resource -> currentRow-1][resource -> currentCol] != 0)  // if element above is not a 0
                            {
                                printf("stuck");  // stuck
                                return;
                            }
                            moveUp(resource, landscape);  // move up 1
                            return;
                        }
                        moveLeft(resource, landscape);  // move left 1
                        return;
                    }
                    moveDown(resource, landscape);  // move down 1
                    return;
                }
            }
    }
    else  // else
    {
        printf("\n (%d, %d) off edges\n", resource -> currentRow, resource -> currentCol);  // off edges
        // possible movements: up, down, left, right

        if ((-1 * rowDifference) > 0)  // if fire is above resource
        {
            if (colDifference < 0)  // if fire is left of resource
            {
                if ((-1 * colDifference) > (-1 * rowDifference))  // if fire is more left than up
                {
                    if (landscape -> grid[resource -> currentRow][resource -> currentCol-1] != 0)  // if element to the left is not a 0
                    {
                        if (landscape -> grid[resource -> currentRow-1][resource -> currentCol] != 0)  // if element above is not a 0
                        {
                            if (landscape -> grid[resource -> currentRow+1][resource -> currentCol] != 0)  // if element below is not a 0
                            {
                                if (landscape -> grid[resource -> currentRow][resource -> currentCol+1] != 0)  // if element to the right is not a 0
                                {
                                    printf("stuck");  // stuck
                                    return;
                                }
                                moveRight(resource, landscape);  // move right 1
                                return;
                            }
                            moveDown(resource, landscape);  // move down 1
                            return;
                        }
                        moveUp(resource, landscape);  // move up 1
                        return;
                    }
                    moveLeft(resource, landscape);  // move left 1
                    return;
                }
                else if ((-1 * rowDifference) >= (-1 * colDifference))  // else if fire is more up than left
                {
                    if (landscape -> grid[resource -> currentRow-1][resource -> currentCol] != 0)  // if element above is not a 0
                    {
                        if (landscape -> grid[resource -> currentRow][resource -> currentCol-1] != 0)  // if element to the left is not a 0
                        {
                            if (landscape -> grid[resource -> currentRow+1][resource -> currentCol] != 0)  // if element below is not a 0
                            {
                                if (landscape -> grid[resource -> currentRow][resource -> currentCol+1] != 0)  // if element to the right is not a 0
                                {
                                    printf("stuck");  // stuck
                                    return;
                                }
                                moveRight(resource, landscape);  // move right 1
                                return;
                            }
                            moveDown(resource, landscape);  // move down 1
                            return;
                        }
                        moveLeft(resource, landscape);  // move left 1
                        return;
                    }
                    moveUp(resource, landscape);  // move up 1
                    return;
                }
            }
            else if (colDifference > 0)  // if fire is right of resource
            {

                if (colDifference > (-1 * rowDifference))  // if fire is more right than up
                {
                    if (landscape -> grid[resource -> currentRow][resource -> currentCol+1] != 0)  // if element to the right is not a 0
                    {
                        if (landscape -> grid[resource -> currentRow-1][resource -> currentCol] != 0)  // if element above is not a 0
                        {
                            if (landscape -> grid[resource -> currentRow+1][resource -> currentCol] != 0)  // if element below is not a 0
                            {
                                if (landscape -> grid[resource -> currentRow][resource -> currentCol-1] != 0)  // if element to the left is not a 0
                                {
                                    printf("stuck");  // stuck
                                    return;
                                }
                                moveLeft(resource, landscape);  // move left 1
                                return;
                            }
                            moveDown(resource, landscape);  // move down 1
                            return;
                        }
                        moveUp(resource, landscape);  // move up 1
                        return;
                    }
                    moveRight(resource, landscape);  // move right 1
                    return;
                }
                else if ((-1 * rowDifference) >= colDifference)  // else if fire is more up than right
                {
                    if (landscape -> grid[resource -> currentRow-1][resource -> currentCol] != 0)  // if element above is not a 0
                    {
                        if (landscape -> grid[resource -> currentRow][resource -> currentCol+1] != 0)  // if element to the right is not a 0
                        {
                            if (landscape -> grid[resource -> currentRow+1][resource -> currentCol] != 0)  // if element below is not a 0
                            {
                                if (landscape -> grid[resource -> currentRow][resource -> currentCol-1] != 0)  // if element to the left is not a 0
                                {
                                    printf("stuck");  // stuck
                                    return;
                                }
                                moveLeft(resource, landscape);  // move left 1
                                return;
                            }
                            moveDown(resource, landscape);  // move down 1
                            return;
                        }
                        moveRight(resource, landscape);  // move right 1
                        return;
                    }
                    moveUp(resource, landscape);  // move up 1
                    return;
                }
            }
            else  // else if fire is on same column and above resource
            {

                if (landscape -> grid[resource -> currentRow-1][resource -> currentCol] != 0)  // if element above is not a 0
                    {
                        if (landscape -> grid[resource -> currentRow][resource -> currentCol+1] != 0)  // if element to the right is not a 0
                        {
                            if (landscape -> grid[resource -> currentRow+1][resource -> currentCol] != 0)  // if element below is not a 0
                            {
                                if (landscape -> grid[resource -> currentRow][resource -> currentCol-1] != 0)  // if element to the left is not a 0
                                {
                                    printf("stuck");  // stuck
                                    return;
                                }
                                moveLeft(resource, landscape);  // move left 1
                                return;
                            }
                            moveDown(resource, landscape);  // move down 1
                            return;
                        }
                        moveRight(resource, landscape);  // move right 1
                        return;
                    }
                    moveUp(resource, landscape);  // move up 1
                    return;

            }
        }
        else if (rowDifference > 0)  // else if fire is below resource
        {
            if (colDifference < 0)  // if fire is left of resource
            {

                if ((-1 * colDifference) > rowDifference)  // if fire is more left than down
                {

                    if (landscape -> grid[resource -> currentRow][resource -> currentCol-1] != 0)  // if element to the left is not a 0
                    {
                        if (landscape -> grid[resource -> currentRow+1][resource -> currentCol] != 0)  // if element below is not a 0
                        {
                            if (landscape -> grid[resource -> currentRow-1][resource -> currentCol] != 0)  // if element above is not a 0
                            {
                                if (landscape -> grid[resource -> currentRow][resource -> currentCol+1] != 0)  // if element to the right is not a 0
                                {
                                    printf("stuck");  // stuck
                                    return;
                                }
                                moveRight(resource, landscape);  // move right 1
                                return;
                            }
                            moveUp(resource, landscape);  // move up 1
                            return;
                        }
                        moveDown(resource, landscape);  // move down 1
                        return;
                    }
                    moveLeft(resource, landscape);  // move left 1
                    return;

                }
                else if (rowDifference >= (-1 * colDifference))  // else if fire is more down than left
                {

                    if (landscape -> grid[resource -> currentRow+1][resource -> currentCol] != 0)  // if element below is not a 0
                    {
                        if (landscape -> grid[resource -> currentRow][resource -> currentCol-1] != 0)  // if element to the left is not a 0
                        {
                            if (landscape -> grid[resource -> currentRow-1][resource -> currentCol] != 0)  // if element above is not a 0
                            {
                                if (landscape -> grid[resource -> currentRow][resource -> currentCol+1] != 0)  // if element to the right is not a 0
                                {
                                    printf("stuck");  // stuck
                                    return;
                                }
                                moveRight(resource, landscape);  // move right 1
                                return;
                            }
                            moveUp(resource, landscape);  // move up 1
                            return;
                        }
                        moveLeft(resource, landscape);  // move left 1
                        return;
                    }
                    moveDown(resource, landscape);  // move down 1
                    return;

                }
            }
            else if (colDifference > 0)  // if fire is right of resource
            {
                if (colDifference > rowDifference)  // if fire is more right than down
                {
                    if (landscape -> grid[resource -> currentRow][resource -> currentCol+1] != 0)  // if element to the right is not a 0
                    {
                        if (landscape -> grid[resource -> currentRow+1][resource -> currentCol] != 0)  // if element below is not a 0
                        {
                            if (landscape -> grid[resource -> currentRow-1][resource -> currentCol] != 0)  // if element above is not a 0
                            {
                                if (landscape -> grid[resource -> currentRow][resource -> currentCol-1] != 0)  // if element to the left is not a 0
                                {
                                    printf("stuck");  // stuck
                                    return;
                                }
                                moveLeft(resource, landscape);  // move left 1
                                return;
                            }
                            moveUp(resource, landscape);  // move up 1
                            return;
                        }
                        moveDown(resource, landscape);  // move down 1
                        return;
                    }
                    moveRight(resource, landscape);  // move right 1
                    return;
                }
                else if(rowDifference >= colDifference)  // else if fire is more down than right

                    if (landscape -> grid[resource -> currentRow+1][resource -> currentCol] != 0)  // if element below is not a 0
                    {
                        if (landscape -> grid[resource -> currentRow][resource -> currentCol+1] != 0)  // if element to the right is not a 0
                        {
                            if (landscape -> grid[resource -> currentRow-1][resource -> currentCol] != 0)  // if element above is not a 0
                            {
                                if (landscape -> grid[resource -> currentRow][resource -> currentCol-1] != 0)  // if element to the left is not 0
                                {
                                    printf("stuck");  // stuck
                                    return;
                                }
                                moveLeft(resource, landscape);  // move left 1
                                return;
                            }
                            moveUp(resource, landscape);  // move up 1
                            return;
                        }
                        moveRight(resource, landscape);  // move right 1
                        return;
                    }
                    moveDown(resource, landscape);  // move down 1
                    return;
            }
        }
        else  // fire is on same column and below resource
        {

            if (landscape -> grid[resource -> currentRow+1][resource -> currentCol] != 0)  // if element below is not a 0
                    {
                        if (landscape -> grid[resource -> currentRow][resource -> currentCol+1] != 0)  // if element to the right is not a 0
                        {
                            if (landscape -> grid[resource -> currentRow-1][resource -> currentCol] != 0)  // if element above is not a 0
                            {
                                if (landscape -> grid[resource -> currentRow][resource -> currentCol-1] != 0)  // if element to the left is not 0
                                {
                                    printf("stuck");  // stuck
                                    return;
                                }
                                moveLeft(resource, landscape);  // move left 1
                                return;
                            }
                            moveUp(resource, landscape);  // move up 1
                            return;
                        }
                        moveRight(resource, landscape);  // move right 1
                        return;
                    }
                    moveDown(resource, landscape);  // move down 1
                    return;

        }
    }
}
void moveLeft(struct Resource *resource, struct Landscape *landscape)
{
    printf("(%d, %d) moving left\n", resource -> currentRow, resource -> currentCol);
    landscape -> grid[resource -> currentRow][resource -> currentCol] = 0;
    resource -> currentCol = resource -> currentCol - 1;
    landscape -> grid[resource -> currentRow][resource -> currentCol] = 7;
    printf("coords after move: (%d, %d)\n", resource -> currentRow, resource -> currentCol);

}
void moveRight(struct Resource *resource, struct Landscape *landscape)
{
    printf("(%d, %d) moving right\n", resource -> currentRow, resource -> currentCol);
    landscape -> grid[resource -> currentRow][resource -> currentCol] = 0;
    resource -> currentCol = resource -> currentCol + 1;
    landscape -> grid[resource -> currentRow][resource -> currentCol] = 7;
    printf("coords after move: (%d, %d)\n", resource -> currentRow, resource -> currentCol);

}
void moveUp(struct Resource *resource, struct Landscape *landscape)
{

    printf("(%d, %d) moving up\n", resource -> currentRow, resource -> currentCol);
    landscape -> grid[resource -> currentRow][resource -> currentCol] = 0;
    resource -> currentRow = resource -> currentRow - 1;
    landscape -> grid[resource -> currentRow][resource -> currentCol] = 7;
    printf("coords after move: (%d, %d)\n", resource -> currentRow, resource -> currentCol);

}
void moveDown(struct Resource *resource, struct Landscape *landscape)
{

    printf("(%d, %d) moving down\n", resource -> currentRow, resource -> currentCol);
    landscape -> grid[resource -> currentRow][resource -> currentCol] = 0;
    resource -> currentRow = resource -> currentRow + 1;
    landscape -> grid[resource -> currentRow][resource -> currentCol] = 7;
    printf("coords after move: (%d, %d)\n", resource -> currentRow, resource -> currentCol);

}
void extinguishFire(struct Resource *resource, struct Landscape *landscape)
{

    printf("(%d, %d) extinguishing fire\n", resource -> currentRow, resource -> currentCol);

    landscape -> grid[resource -> assignedFireRow][resource -> assignedFireCol] = 0;

    for (int i = 0; i < GLOBAL_RESOURCES; i++)
            {

                if (assignedResources[i].currentRow == resource -> currentRow && assignedResources[i].currentCol == resource -> currentCol)
                {

                    for (int j = 0; j < GLOBAL_RESOURCES; j++)
                    {
                        if (freeResources[j].currentRow == -1)
                        {

                                freeResources[j].currentRow = assignedResources[i].currentRow;
                                freeResources[j].currentCol = assignedResources[i].currentCol;
                                break;

                        }

                    }
                    for (int j = i; j < GLOBAL_RESOURCES - 1; j++)
                    {

                        assignedResources[j] = assignedResources[j+1];

                    }
                    assignedResources[GLOBAL_RESOURCES-1].currentCol = -1;
                    assignedResources[GLOBAL_RESOURCES-1].currentRow = -1;
                    assignedResources[GLOBAL_RESOURCES-1].assignedFireCol = -1;
                    assignedResources[GLOBAL_RESOURCES-1].assignedFireRow = -1;
                    break;
                }
                else
                {
                    continue;
                }

            }

}

int main() {
  // Create pipes for IPC
  int pipe_fd1[2];
  int pipe_fd2[2];
  // Initialize first set of pipes
  if (pipe(pipe_fd1) == -1) {
    perror("pipe");
    exit(EXIT_FAILURE);
  }
  // Initialize second set of pipes
  if (pipe(pipe_fd2) == -1) {
    perror("pipe");
    exit(EXIT_FAILURE);
  }

  struct Landscape *landscape = (struct Landscape *)malloc(sizeof(struct Landscape));

  if (landscape == NULL) {
      fprintf(stderr, "Memory allocation failed\n");
      return 1;
  }

  initResourceArrays();

  // Define number of rows
  landscape -> rows = 30;
  // Define number of columns
  landscape -> columns = 30;
  // Create array to represent the landscape
  int landscapeGrid[landscape -> rows][landscape -> columns];
  // setting the array to the landscape struct
  landscape -> grid = landscapeGrid;
  int totalGridRows = landscape -> rows;
  int totalGridColumns = landscape -> columns;
  // Fill array with 0s
  for (int i = 0; i < totalGridRows; i++)
  {
      for (int j = 0; j < totalGridColumns; j++)
      {
        if (i == totalGridRows / 2)
        {
            landscape -> grid[i][j] = 5;
            if (j == totalGridColumns / 2)
            {
              landscape -> grid[i][j] = 5;
              continue;
            }
        }
        landscape -> grid[i][j] = 0;
      }
  }

  placeResources(landscape);

  // Number of fires to be set.
  int numOfFires = 5;
  // Create array of Fire structs to store info on each fire set.
  struct Fire fires[numOfFires];
  // For loop to set the fires.
  for (int j = 0; j < numOfFires; j++) {
    // Stores attributes (row, col, , number of required resources) about the fire in jth element of Fire struct array.
    fires[j] = setFire(landscape);
    printf("\n");
  }

  printf("\nFires set\n");
  printLandscape(landscape);

  // Now that all fires are set, create a child process to search for fires
  pid_t child_pid = fork();
  if (child_pid == 0) {
    // Search for fires in the landscape. If one is found, use IPC to communicate that with the parent process.
    // Close read end of pipe
    close(pipe_fd1[0]);

    int numOfThreads = 4;

    // Initialize semaphore
    sem_init(&semaphore, 0, 1);
    // Thread IDs
    pthread_t threads[numOfThreads];
    struct ThreadArgs threadArgs[numOfThreads];

    for (int i = 0; i < numOfThreads; i++)
    {

        threadArgs[i].beginRow = 0 + ((totalGridRows / numOfThreads) * i);
        if (i == numOfThreads - 1)
        {
            threadArgs[i].endRow = totalGridRows;
        }
        else
        {
        threadArgs[i].endRow = (totalGridRows / numOfThreads) * (i+1);
        }
        threadArgs[i].landscape = landscape;
        threadArgs[i].fireToDispatch = pipe_fd1[1];

    }

    // Have threads search for fires in the landscape

    for (int i = 0; i < numOfThreads; i++)
    {

        pthread_create(&threads[i], NULL, searchForFires, &threadArgs[i]);

    }

    // Wait for threads to finish

    for (int i = 0; i < numOfThreads; i++)
    {

        pthread_join(threads[i], NULL);

    }

    printf("Threads finished\n");

    // After child finishes search of landscape, notify the parent

    int complete = -1;

    for (int i = 0; i < numOfThreads; i++)
    {

        write(pipe_fd1[1], &complete, sizeof(-1));

    }

    // Close write end of pipe
    close(pipe_fd1[1]);

  } else if (child_pid > 0) {
    // Parent process
    // r stores row and c stores column
    int spiralScanResults[2] = {0};
    int fireCoords[2];
    // Close write end of pipe
    close(pipe_fd1[1]);

    // While child process searches for fires, listen for IPC from child process
    while (true)
    {

      if (read(pipe_fd1[0], &fireCoords, sizeof(fireCoords)) != 0)
      {
        if (fireCoords[0] != -1 && fireCoords[1] != -1)
        {
            spiralScanForFreeResource(landscape, fireCoords[0], fireCoords[1], spiralScanResults);
            for (int i = 0; i < GLOBAL_RESOURCES; i++)
            {

                if (freeResources[i].currentRow == spiralScanResults[0] && freeResources[i].currentCol == spiralScanResults[1])
                {

                    for (int j = 0; j < GLOBAL_RESOURCES; j++)
                    {
                        if (assignedResources[j].currentRow == -1)
                        {

                                assignedResources[j].currentRow = freeResources[i].currentRow;
                                assignedResources[j].currentCol = freeResources[i].currentCol;
                                assignedResources[j].assignedFireRow = fireCoords[0];
                                assignedResources[j].assignedFireCol = fireCoords[1];
                                break;

                        }

                    }
                    for (int j = i; j < GLOBAL_RESOURCES - 1; j++)
                    {

                        freeResources[j] = freeResources[j+1];

                    }
                    freeResources[GLOBAL_RESOURCES-1].currentCol = -1;
                    freeResources[GLOBAL_RESOURCES-1].currentRow = -1;
                    freeResources[GLOBAL_RESOURCES-1].assignedFireCol = -1;
                    freeResources[GLOBAL_RESOURCES-1].assignedFireRow = -1;
                    break;
                }
                else
                {
                    continue;
                }

            }
        }
        else
        {
            // Child process has finished searching for fires
            break;
        }
      }

    }
    // Wait to ensure child process has finished executing.
    wait(NULL);

    for (int i = 0; i < GLOBAL_RESOURCES; i++)
    {

        printf("Resource %d location: (%d, %d), Assigned Fire location: (%d, %d)\n", i+1, freeResources[i].currentRow, freeResources[i].currentCol, freeResources[i].assignedFireRow, freeResources[i].assignedFireCol);

    }
    for (int i = 0; i < GLOBAL_RESOURCES; i++)
    {

        printf("Resource %d location: (%d, %d), Assigned Fire location: (%d, %d)\n", i+1, assignedResources[i].currentRow, assignedResources[i].currentCol, assignedResources[i].assignedFireRow, assignedResources[i].assignedFireCol);

    }

    for (int i = 0; i < 30; i++)
    {
        for (int j = 0; j < GLOBAL_RESOURCES; j++)
        {
            if (assignedResources[j].currentRow == -1)
            {
                continue;
            }
            moveResource(&assignedResources[j], landscape);
        }
        printf("\n");
        printLandscape(landscape);
        sleep(0.5);
    }


    printf("\n");
    printLandscape(landscape);


  } else {
    perror("Failed to create child process.");
    exit(EXIT_FAILURE);
  }

  // Destroy semaphore
  sem_destroy(&semaphore);

  // Free the memory allocated to the landscape struct
  free(landscape);
  return 0;
}

Overwriting cmpsc472Project2.c


In [38]:
%%shell
gcc cmpsc472Project2.c -o cmpsc472Project2
./cmpsc472Project2

new resource at position (2,13)
new resource at position (27,28)
new resource at position (20,26)
new resource at position (2,1)
new resource at position (13,24)
new resource at position (6,16)
new resource at position (17,8)
new resource at position (13,19)
new resource at position (15,21)
new resource at position (6,11)
new fire at position (2,9)

new fire at position (10,16)

new fire at position (21,5)

new fire at position (29,0)

new fire at position (7,4)


Fires set
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 7 0 0 0 0 0 0 0 1 0 0 0 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 7 0 0 0 0 7 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0